# Libraries

In [1]:
import numpy as np
import pandas as pd
import pickle
import tqdm.auto as tqdm
from pprint import pprint
import collections
import random
import feather

# Use restricted_wcc dataset (only courses from WCC, no summer, keep only row from last five years or from a student with a course in the last five years) from wrangling_restricted_wcc_dataset.ipynb

In [2]:
WCC_PD_PATH = "../data/wcc_pd.pkl"

with open(WCC_PD_PATH, "rb") as file:
    raw_data = pickle.load(file)
    
raw_data

,acad_career,strm,stdnt_enrl_status,unt_taken,unt_billing,crse_grade_input,earn_credit,emplid,subject,catalog_nbr,crse_acad_org,sex,gpa,grade_points,total_units,cum_grade_points,cum_units,overall_gpa,prior_term_gpa,course_name
0,UG,1182.0,E,5.0,5.0,A,Y,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,MATH,51,MATH,M,4.0,72.0,18.0,72.0,18.0,4.000000,NaN,MATH51
1,UG,1182.0,E,0.0,0.0,None,N,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,CHEM,31X,CHEMISTRY,M,NaN,72.0,18.0,72.0,18.0,4.000000,NaN,CHEM31X
2,UG,1182.0,E,5.0,5.0,A,Y,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,CHEM,31X,CHEMISTRY,M,4.0,72.0,18.0,72.0,18.0,4.000000,NaN,CHEM31X
3,UG,1182.0,E,0.0,0.0,None,N,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,MATH,51,MATH,M,NaN,72.0,18.0,72.0,18.0,4.000000,NaN,MATH51
4,UG,1184.0,E,5.0,5.0,A,Y,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,CS,106X,COMPUTSCI,M,4.0,65.1,17.0,137.1,35.0,3.917143,4.0,CS106X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784350,UG,1202.0,E,5.0,5.0,B,Y,ZZCaHx+CP6JrUW8AJ/N+m9uPlGbsOEaeCC8uU50l/iVr9d...,CHEM,31A,CHEMISTRY,F,3.0,39.0,11.0,39.0,11.0,3.545455,NaN,CHEM31A
784351,UG,1202.0,E,0.0,0.0,None,N,ZZCaHx+CP6JrUW8AJ/N+m9uPlGbsOEaeCC8uU50l/iVr9d...,CHEM,31A,CHEMISTRY,F,NaN,39.0,11.0,39.0,11.0,3.545455,NaN,CHEM31A
784352,UG,1204.0,E,5.0,5.0,None,Y,ZZCaHx+CP6JrUW8AJ/N+m9uPlGbsOEaeCC8uU50l/iVr9d...,PSYCH,1,PSYCHOLOGY,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PSYCH1
784353,UG,1204.0,E,5.0,5.0,None,Y,ZZCaHx+CP6JrUW8AJ/N+m9uPlGbsOEaeCC8uU50l/iVr9d...,CHEM,31B,CHEMISTRY,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEM31B


In [3]:
all_courses = set(raw_data["course_name"])
pprint(all_courses)
print(len(all_courses))

{'AA100',
 'AA200',
 'AA210A',
 'AA212',
 'AA214A',
 'AA214B',
 'AA228',
 'AA229',
 'AA240A',
 'AA240B',
 'AA241A',
 'AA241B',
 'AA241X',
 'AA242A',
 'AA256',
 'AA270',
 'AA271A',
 'AA272C',
 'AA279A',
 'AA279B',
 'AA280',
 'APPPHYS220',
 'APPPHYS272',
 'APPPHYS273',
 'BIO101',
 'BIO104',
 'BIO107',
 'BIO115',
 'BIO144',
 'BIO153',
 'BIO156',
 'BIO158',
 'BIO163',
 'BIO173',
 'BIO188',
 'BIO189',
 'BIO234',
 'BIO239',
 'BIO256',
 'BIO258',
 'BIO41',
 'BIO42',
 'BIO43',
 'BIO44X',
 'BIO44Y',
 'BIOE101',
 'BIOE103',
 'BIOE103B',
 'BIOE123',
 'BIOE141A',
 'BIOE141B',
 'BIOE201C',
 'BIOE210',
 'BIOE211',
 'BIOE212',
 'BIOE244',
 'BIOE280',
 'BIOE300A',
 'BIOE301C',
 'BIOE311',
 'BIOE313',
 'BIOE355',
 'BIOE41',
 'BIOE42',
 'BIOE44',
 'BIOE454',
 'BIOHOPK163H',
 'BIOHOPK167H',
 'BIOHOPK263H',
 'BIOMEDIN156',
 'BIOMEDIN210',
 'BIOMEDIN212',
 'BIOMEDIN214',
 'BIOMEDIN215',
 'BIOMEDIN217',
 'BIOMEDIN218',
 'BIOMEDIN224',
 'BIOMEDIN225',
 'BIOMEDIN256',
 'BIOMEDIN260',
 'BIOMEDIN371',
 'BIOPHYS

 'STATS200',
 'STATS202',
 'STATS203',
 'STATS206',
 'STATS207',
 'STATS215',
 'STATS217',
 'STATS219',
 'STATS221',
 'STATS231',
 'STATS240',
 'STATS244',
 'STATS250',
 'STATS263',
 'STATS267',
 'STATS270',
 'STATS271',
 'STATS290',
 'STATS300A',
 'STATS305',
 'STATS306A',
 'STATS310A',
 'STATS310B',
 'STATS310C',
 'STATS311',
 'STATS315A',
 'STATS316',
 'STATS344',
 'STATS350',
 'STATS360',
 'STATS363',
 'STATS370',
 'STATS371',
 'STATS374',
 'STATS376A',
 'STATS50',
 'STATS60',
 'SYMSYS100'}
1012


In [4]:
# descriptive statistics
print(len(raw_data), "unique course records")
print(len(raw_data["emplid"].unique()), "unique students")
print(len(raw_data["course_name"].unique()), "unique courses")

317636 unique course records
13709 unique students
1012 unique courses


In [5]:
OFFICIAL_PREREQ_PATH = "../data/official_reqs.pkl"

with open(OFFICIAL_PREREQ_PATH, "rb") as file:
    official_prereqs = pickle.load(file)
    
pprint(official_prereqs["CS110"])
pprint(official_prereqs["CS221"])
pprint(official_prereqs)

['CS107']
['CS103', 'CS106B', 'CS106X', 'CS107', 'CS109']
{'AA100': ['MATH20', 'MATH21', 'MATH41', 'MATH42'],
 'AA102': [],
 'AA103': [],
 'AA108N': [],
 'AA109Q': [],
 'AA113N': [],
 'AA115N': [],
 'AA115Q': [],
 'AA116N': [],
 'AA116Q': [],
 'AA118N': [],
 'AA119N': [],
 'AA120Q': [],
 'AA121Q': [],
 'AA122N': [],
 'AA131': [],
 'AA135': [],
 'AA136A': [],
 'AA136B': [],
 'AA141': [],
 'AA146A': [],
 'AA146B': [],
 'AA149': [],
 'AA151': [],
 'AA156': [],
 'AA173': [],
 'AA174A': [],
 'AA174B': [],
 'AA190': [],
 'AA199': [],
 'AA200': ['AA210A'],
 'AA200A': [],
 'AA200B': [],
 'AA201A': [],
 'AA201B': ['AA201A'],
 'AA202': [],
 'AA203': [],
 'AA204': [],
 'AA208': ['AA210A'],
 'AA210A': [],
 'AA210B': ['AA210A'],
 'AA212': ['ENGR205', 'EE263'],
 'AA214': [],
 'AA214A': ['CME108'],
 'AA214B': ['CME206'],
 'AA214C': ['AA214B'],
 'AA215A': ['AA214B'],
 'AA215B': ['AA214B'],
 'AA218': [],
 'AA222': [],
 'AA228': [],
 'AA229': ['AA228', 'CS238', 'CS221'],
 'AA236A': [],
 'AA236B': ['AA23

 'AMELANG108C': ['AMELANG108B'],
 'AMELANG110A': [],
 'AMELANG110B': [],
 'AMELANG110C': [],
 'AMELANG114A': [],
 'AMELANG114B': [],
 'AMELANG115A': [],
 'AMELANG115B': [],
 'AMELANG115C': [],
 'AMELANG117A': [],
 'AMELANG117B': [],
 'AMELANG117C': [],
 'AMELANG118A': [],
 'AMELANG118B': [],
 'AMELANG118C': [],
 'AMELANG11A': [],
 'AMELANG11B': [],
 'AMELANG11C': [],
 'AMELANG11G': [],
 'AMELANG11H': [],
 'AMELANG120A': [],
 'AMELANG120B': [],
 'AMELANG120C': [],
 'AMELANG121A': [],
 'AMELANG121B': [],
 'AMELANG121C': [],
 'AMELANG122A': [],
 'AMELANG122B': [],
 'AMELANG122C': [],
 'AMELANG126': [],
 'AMELANG127': [],
 'AMELANG128A': [],
 'AMELANG128B': ['AMELANG128A'],
 'AMELANG128C': ['AMELANG128B'],
 'AMELANG129A': ['AMELANG128C'],
 'AMELANG129B': ['AMELANG129A'],
 'AMELANG129C': ['AMELANG129B'],
 'AMELANG12G': [],
 'AMELANG12H': [],
 'AMELANG12S': [],
 'AMELANG130A': ['AMELANG129C'],
 'AMELANG130B': [],
 'AMELANG130C': [],
 'AMELANG131': [],
 'AMELANG131A': [],
 'AMELANG131B': [],


 'ANTHRO100B': [],
 'ANTHRO100C': [],
 'ANTHRO100D': [],
 'ANTHRO101': [],
 'ANTHRO101A': [],
 'ANTHRO101B': [],
 'ANTHRO101S': [],
 'ANTHRO102': [],
 'ANTHRO102A': [],
 'ANTHRO102B': [],
 'ANTHRO102C': [],
 'ANTHRO103': [],
 'ANTHRO103A': [],
 'ANTHRO104': [],
 'ANTHRO104A': [],
 'ANTHRO105': [],
 'ANTHRO105A': [],
 'ANTHRO105B': [],
 'ANTHRO106': [],
 'ANTHRO106A': [],
 'ANTHRO107A': [],
 'ANTHRO108A': [],
 'ANTHRO108B': [],
 'ANTHRO108E': [],
 'ANTHRO109A': [],
 'ANTHRO10A': [],
 'ANTHRO10SC': [],
 'ANTHRO110': [],
 'ANTHRO110A': [],
 'ANTHRO110B': [],
 'ANTHRO111': [],
 'ANTHRO111A': [],
 'ANTHRO111B': [],
 'ANTHRO111C': [],
 'ANTHRO112': [],
 'ANTHRO112A': [],
 'ANTHRO112B': ['ANTHRO112'],
 'ANTHRO113': [],
 'ANTHRO113B': [],
 'ANTHRO114': [],
 'ANTHRO114A': [],
 'ANTHRO114B': [],
 'ANTHRO115': [],
 'ANTHRO115A': [],
 'ANTHRO115B': [],
 'ANTHRO116': [],
 'ANTHRO116A': [],
 'ANTHRO116B': [],
 'ANTHRO116C': [],
 'ANTHRO117': [],
 'ANTHRO117A': [],
 'ANTHRO117B': [],
 'ANTHRO117C': [

 'ANTHRO365B': [],
 'ANTHRO366': [],
 'ANTHRO366W': [],
 'ANTHRO367': [],
 'ANTHRO367B': [],
 'ANTHRO368': [],
 'ANTHRO368A': [],
 'ANTHRO369': [],
 'ANTHRO370': [],
 'ANTHRO371': [],
 'ANTHRO371B': [],
 'ANTHRO372': [],
 'ANTHRO372A': [],
 'ANTHRO373': [],
 'ANTHRO374': [],
 'ANTHRO376': [],
 'ANTHRO377': [],
 'ANTHRO378A': [],
 'ANTHRO378B': [],
 'ANTHRO379': [],
 'ANTHRO379A': [],
 'ANTHRO379B': [],
 'ANTHRO380': [],
 'ANTHRO380A': [],
 'ANTHRO380B': [],
 'ANTHRO380C': [],
 'ANTHRO381': [],
 'ANTHRO382': [],
 'ANTHRO384': [],
 'ANTHRO385': [],
 'ANTHRO386': [],
 'ANTHRO387': [],
 'ANTHRO388': [],
 'ANTHRO389': [],
 'ANTHRO39': [],
 'ANTHRO390': [],
 'ANTHRO391': [],
 'ANTHRO393': [],
 'ANTHRO397A': [],
 'ANTHRO397B': [],
 'ANTHRO398': [],
 'ANTHRO398B': [],
 'ANTHRO399': [],
 'ANTHRO4': [],
 'ANTHRO400': [],
 'ANTHRO401A': [],
 'ANTHRO401B': [],
 'ANTHRO41': [],
 'ANTHRO42': [],
 'ANTHRO440': [],
 'ANTHRO441': [],
 'ANTHRO442': [],
 'ANTHRO444': [],
 'ANTHRO445': [],
 'ANTHRO446A': 

 'ARCHLGY80': [],
 'ARCHLGY81': [],
 'ARCHLGY85': [],
 'ARCHLGY92': [],
 'ARCHLGY97': [],
 'ARCHLGY98B': [],
 'ARCHLGY99A': [],
 'ARTHIST1': [],
 'ARTHIST100N': [],
 'ARTHIST101': [],
 'ARTHIST102': [],
 'ARTHIST103S': [],
 'ARTHIST104A': [],
 'ARTHIST105': [],
 'ARTHIST105A': [],
 'ARTHIST105B': [],
 'ARTHIST106': [],
 'ARTHIST107': [],
 'ARTHIST107A': [],
 'ARTHIST107Y': [],
 'ARTHIST108A': [],
 'ARTHIST108B': [],
 'ARTHIST109': [],
 'ARTHIST109D': [],
 'ARTHIST10SC': [],
 'ARTHIST11': [],
 'ARTHIST110': [],
 'ARTHIST110Y': [],
 'ARTHIST111': [],
 'ARTHIST111A': [],
 'ARTHIST111Y': [],
 'ARTHIST112Y': [],
 'ARTHIST113': [],
 'ARTHIST113A': [],
 'ARTHIST113Y': [],
 'ARTHIST114': [],
 'ARTHIST115': [],
 'ARTHIST116': [],
 'ARTHIST117': [],
 'ARTHIST118': [],
 'ARTHIST118A': [],
 'ARTHIST118N': [],
 'ARTHIST119': [],
 'ARTHIST11SC': [],
 'ARTHIST120': [],
 'ARTHIST121': [],
 'ARTHIST121S': [],
 'ARTHIST122': [],
 'ARTHIST123N': [],
 'ARTHIST123Y': [],
 'ARTHIST124': [],
 'ARTHIST125': [

...

 'SOC207': [],
 'SOC208': [],
 'SOC209': [],
 'SOC20N': [],
 'SOC210': [],
 'SOC211': [],
 'SOC212': [],
 'SOC213A': [],
 'SOC214': [],
 'SOC215': [],
 'SOC216': [],
 'SOC217A': [],
 'SOC217B': [],
 'SOC218': [],
 'SOC219': [],
 'SOC220': [],
 'SOC221VP': [],
 'SOC223': [],
 'SOC224A': [],
 'SOC224B': [],
 'SOC224VP': [],
 'SOC225A': [],
 'SOC226': [],
 'SOC227': [],
 'SOC228': [],
 'SOC229': [],
 'SOC229X': [],
 'SOC22N': [],
 'SOC230': [],
 'SOC231': [],
 'SOC232': [],
 'SOC234': [],
 'SOC235': [],
 'SOC236': [],
 'SOC236A': [],
 'SOC236B': [],
 'SOC237': [],
 'SOC238': [],
 'SOC239': ['SOC138'],
 'SOC240': [],
 'SOC240A': [],
 'SOC240W': [],
 'SOC241': [],
 'SOC241A': [],
 'SOC241B': [],
 'SOC241P': [],
 'SOC242': [],
 'SOC243': [],
 'SOC244': [],
 'SOC245': [],
 'SOC247': [],
 'SOC247A': [],
 'SOC247B': [],
 'SOC247C': [],
 'SOC248': [],
 'SOC249': [],
 'SOC249X': [],
 'SOC24N': [],
 'SOC250': [],
 'SOC251': [],
 'SOC252': [],
 'SOC253': [],
 'SOC254': [],
 'SOC254C': [],
 'SOC255'

 'SPECLANG115B': [],
 'SPECLANG115C': [],
 'SPECLANG119A': [],
 'SPECLANG119B': [],
 'SPECLANG123A': [],
 'SPECLANG123B': [],
 'SPECLANG123C': [],
 'SPECLANG125A': [],
 'SPECLANG125B': [],
 'SPECLANG125C': [],
 'SPECLANG126A': [],
 'SPECLANG126B': [],
 'SPECLANG129A': [],
 'SPECLANG129B': ['SPECLANG129A'],
 'SPECLANG129C': ['SPECLANG129B'],
 'SPECLANG130A': ['SPECLANG129C'],
 'SPECLANG130B': ['SPECLANG130A'],
 'SPECLANG130C': ['SPECLANG130B'],
 'SPECLANG131A': ['SPECLANG130C'],
 'SPECLANG131B': [],
 'SPECLANG131C': [],
 'SPECLANG132A': [],
 'SPECLANG132B': [],
 'SPECLANG132C': [],
 'SPECLANG133': [],
 'SPECLANG134A': [],
 'SPECLANG136A': [],
 'SPECLANG136B': [],
 'SPECLANG136C': [],
 'SPECLANG138A': [],
 'SPECLANG138B': ['SPECLANG138A'],
 'SPECLANG138C': ['SPECLANG138B'],
 'SPECLANG139A': [],
 'SPECLANG139B': [],
 'SPECLANG144A': [],
 'SPECLANG144B': [],
 'SPECLANG144C': ['SPECLANG144B'],
 'SPECLANG145A': ['SPECLANG144C'],
 'SPECLANG145B': ['SPECLANG145A'],
 'SPECLANG145C': ['SPECLANG1

 'STRAMGT503': [],
 'STRAMGT504': [],
 'STRAMGT508': [],
 'STRAMGT509': [],
 'STRAMGT510': [],
 'STRAMGT511': [],
 'STRAMGT512': [],
 'STRAMGT513': [],
 'STRAMGT514': [],
 'STRAMGT516': [],
 'STRAMGT517': [],
 'STRAMGT518': [],
 'STRAMGT519': [],
 'STRAMGT520': [],
 'STRAMGT524': [],
 'STRAMGT526': [],
 'STRAMGT527': [],
 'STRAMGT529': [],
 'STRAMGT531': [],
 'STRAMGT532': [],
 'STRAMGT533': [],
 'STRAMGT535': [],
 'STRAMGT536': [],
 'STRAMGT537': [],
 'STRAMGT538': [],
 'STRAMGT539': [],
 'STRAMGT542': [],
 'STRAMGT543': [],
 'STRAMGT544': [],
 'STRAMGT545': [],
 'STRAMGT546': [],
 'STRAMGT547': [],
 'STRAMGT549': [],
 'STRAMGT550': [],
 'STRAMGT552': [],
 'STRAMGT554': [],
 'STRAMGT555': [],
 'STRAMGT556': [],
 'STRAMGT559': [],
 'STRAMGT562': [],
 'STRAMGT565': [],
 'STRAMGT567': [],
 'STRAMGT573': [],
 'STRAMGT574': [],
 'STRAMGT577': [],
 'STRAMGT579': [],
 'STRAMGT582': [],
 'STRAMGT583': [],
 'STRAMGT584': [],
 'STRAMGT585': [],
 'STRAMGT586': [],
 'STRAMGT587': [],
 'STRAMGT588

 'TAPS176N': [],
 'TAPS176S': [],
 'TAPS177': [],
 'TAPS177C': [],
 'TAPS177W': [],
 'TAPS178': [],
 'TAPS178B': [],
 'TAPS178C': [],
 'TAPS178D': [],
 'TAPS178E': [],
 'TAPS179': [],
 'TAPS179C': [],
 'TAPS179F': [],
 'TAPS179G': [],
 'TAPS17AX': [],
 'TAPS17N': [],
 'TAPS180P': [],
 'TAPS180Q': [],
 'TAPS181Q': [],
 'TAPS183C': [],
 'TAPS183E': [],
 'TAPS184C': [],
 'TAPS184Q': [],
 'TAPS189': [],
 'TAPS190': [],
 'TAPS191': [],
 'TAPS192': [],
 'TAPS193': [],
 'TAPS197': [],
 'TAPS19AX': [],
 'TAPS19N': [],
 'TAPS20': [],
 'TAPS200': [],
 'TAPS201': [],
 'TAPS201A': [],
 'TAPS201B': [],
 'TAPS201C': [],
 'TAPS201D': [],
 'TAPS202': [],
 'TAPS203': [],
 'TAPS20A': [],
 'TAPS20AX': [],
 'TAPS20N': [],
 'TAPS21': [],
 'TAPS210V': [],
 'TAPS213': ['TAPS103'],
 'TAPS21AR': [],
 'TAPS21N': [],
 'TAPS21S': [],
 'TAPS21T': [],
 'TAPS22': [],
 'TAPS220': [],
 'TAPS220A': [],
 'TAPS223': [],
 'TAPS22N': [],
 'TAPS23': [],
 'TAPS231': ['TAPS131'],
 'TAPS232': ['TAPS132'],
 'TAPS233': ['TAPS133

 'WELLNESS190': [],
 'WELLNESS191': [],
 'WELLNESS192': [],
 'WELLNESS193': [],
 'WELLNESS194': [],
 'WELLNESS195': [],
 'WELLNESS196': [],
 'WELLNESS197': [],
 'WELLNESS198': [],
 'WELLNESS199': [],
 'WELLNESS200': [],
 'WELLNESS201': [],
 'WELLNESS202': [],
 'WELLNESS204': [],
 'WELLNESS205': [],
 'WELLNESS206': [],
 'WELLNESS207': [],
 'WELLNESS208': [],
 'WELLNESS210': [],
 'WELLNESS211': [],
 'WELLNESS212': [],
 'WELLNESS214': [],
 'WELLNESS215': [],
 'WELLNESS217': [],
 'WELLNESS219': [],
 'WELLNESS220': [],
 'WELLNESS230': [],
 'WELLNESS233': [],
 'WELLNESS234': [],
 'WELLNESS235': [],
 'WELLNESS250': [],
 'WELLNESS254': [],
 'WELLNESS255': [],
 'WELLNESS260': [],
 'WELLNESS261': [],
 'WELLNESS263': [],
 'WELLNESS280': [],
 'WELLNESS281': [],
 'WELLNESS282': [],
 'WELLNESS290': [],
 'WELLNESS291': [],
 'WELLNESS293': [],
 'WELLNESS299': [],
 'WELLNESS301': []}


In [6]:
# Builds the feature matrix X and output vector y for a given course, using all other courses in the dataset
# as possible predictors. Only adds a column for another course if at least one student has taken that other
# course before the given course.
# Set gender = True to also add a prediction vector for female (0/1).
# Returns a dataframe with y appended to the right of X, split them off later in get_and_split_data().
############################################################################################################
# 2020-02-23 Initial model settings: 
# Predict success in a course (B+ or better) based on having passed other courses previously (D- or better)
# Incomplete data for strm 1204 (2019-2020 Winter)
# Only consider courses taken at or after strm 1162 (2015-2016 Autumn)
# Y1GRADES = ["A+", "A", "A-", "B+"]
# X1GRADES = ["A+", "A", "A-", "B+", "B", "B-", 
#             "C+", "C", "C-", "D+", "D", "D-"]
# UNFINISHED_QUARTERS = [1204]
# CUTOFF_QUARTER = 1162

# Y1GRADES = ["A+", "A", "A-", "B+"] # we encode these grades as y = 1, otherwise y = 0
# X1GRADES = ["A+", "A", "A-", "B+", "B", "B-", 
#             "C+", "C", "C-", "D+", "D", "D-"] # for previous courses, encode these grades as X_i = 1
UNFINISHED_QUARTERS = [1204] # unfinished quarter(s) if pulled during a quarter
CUTOFF_QUARTER = 1162

# added x1param and y1param as optional parameters, use these or custom lists
LETTER_GRADES = ["A+", "A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D+", "D", "D-", "NP", "W"]
PASS_GRADES = ["A+", "A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D+", "D", "D-"]
SUCCESS_GRADES = ["A+", "A", "A-", "B+"]

def getStudentFeatures(target_course, gender = False, gpa = False, prereqSuccess = False, x1param = PASS_GRADES, y1param = SUCCESS_GRADES, prereqs = False):
    if prereqSuccess:
        x1param = SUCCESS_GRADES
    course_entries = raw_data.loc[(raw_data['course_name'] == target_course) & # all course entries that are about the certain course we want
                                  (raw_data['crse_grade_input'].isin(LETTER_GRADES)) & # Grade needs to be a letter
                                  (raw_data['stdnt_enrl_status'] != 'D') & # not dropped
                                  (raw_data['strm'] >= CUTOFF_QUARTER) & # target (outcome) course during or after the cutoff
                                  (~raw_data['strm'].isin(UNFINISHED_QUARTERS))] # we don't have outcomes for these
    students = course_entries.emplid # Students from the course entries
    students_prev_classes = raw_data.loc[raw_data['emplid'].isin(students)] # Other classes taken by these students
    possible_prev_classes = students_prev_classes['course_name'].unique().tolist() # List of courses taken by these students
    possible_prev_classes.append("strm") # For now add term to the data of when the course was taken
    if gender:
        possible_prev_classes.append("female") # Add gender
    if gpa: # Add GPA
        possible_prev_classes.append("gpa_A")
        possible_prev_classes.append("gpa_B")
        possible_prev_classes.append("gpa_C")
        possible_prev_classes.append("gpa_D")
        possible_prev_classes.append("gpa_NP")
        possible_prev_classes.append("gpa_NaN")
    possible_prev_classes.append("emplid") # Add emplid
    possible_prev_classes.append("y") # Add the target to the data. Will split it later
    a = np.zeros(shape = (len(course_entries), len(possible_prev_classes))) # all 0 initialized features matrix
    full_matrix = pd.DataFrame(a, columns = possible_prev_classes) # same as above, but in pandas to allow indexing by course name
    for counter, idx in enumerate(course_entries.index): # loop over all class entries
        grade = course_entries.loc[[idx]]['crse_grade_input']
        gradeStr = grade.tolist()[0]
        if (gradeStr in y1param):
            full_matrix.iloc[counter, full_matrix.columns.get_loc("y")] = 1
        # fill predictor matrix X
        std_id = course_entries.loc[[idx]]['emplid'] # Get the student id
        quarter = course_entries.loc[[idx]]['strm'] # Get the term
        student_courses = students_prev_classes.loc[students_prev_classes['emplid'] == std_id.tolist()[0]] # Get all courses the student took
        student_courses = student_courses.loc[student_courses['strm'] < quarter.tolist()[0]] # Filter the courses to be only before the course of interest
        if gender:
            female = course_entries.loc[[idx]]['sex'] # Get the student gender
            if (female.tolist()[0] == 'M'):
                full_matrix.iloc[counter, full_matrix.columns.get_loc("female")] = 0
            else:
                full_matrix.iloc[counter, full_matrix.columns.get_loc("female")] = 1
        if gpa:
            prior_GPA = course_entries.loc[[idx]]['prior_term_gpa'] # Get the student's GPA for the previous term
            full_matrix.iloc[counter, full_matrix.columns.get_loc(getTargetGPAColumn(prior_GPA.tolist()[0]))] = 1
        full_matrix.iloc[counter, full_matrix.columns.get_loc("strm")] = quarter.tolist()[0]
        full_matrix.iloc[counter, full_matrix.columns.get_loc("emplid")] = std_id.tolist()[0]
        for j in student_courses.index:# Loop over the student's courses
            course_name = student_courses.loc[[j]]['subject']+student_courses.loc[[j]]['catalog_nbr'] # Get the course name
            grade = student_courses.loc[[j]]['crse_grade_input']
            gradeStr = grade.tolist()[0]
            if (gradeStr in x1param):
                full_matrix.iloc[counter, full_matrix.columns.get_loc(course_name.tolist()[0])] = 1 # Set the course name in X to the new value (or if it was already passed keep the value)
    if not prereqs:
        return full_matrix
    
    # compute prereq_matrix with official prereq pickle
    prereq_columns = official_prereqs[target_course] + ['strm', 'emplid', 'y']
    if gender:
        prereq_columns.extend(['female'])
    if gpa:
        prereq_columns.extend(['gpa_A', 'gpa_B', 'gpa_C', 'gpa_D', 'gpa_NP', 'gpa_NaN'])
    prereq_matrix = full_matrix[prereq_columns]
    return full_matrix, prereq_matrix

In [7]:
features_cs110_testparams = getStudentFeatures('CS110', x1param = PASS_GRADES, y1param = PASS_GRADES)
features_cs110_testparams

,MATH51,CHEM31X,CS106X,CHEM33,PHIL1,CS103,CHEM35,CS109,CS161,CS107,...,PHYSICS230,PHYSICS231,BIO153,MATH220,PHYSICS62,ME105,AA272C,strm,emplid,y
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1202.0,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1192.0,$2a$15$5tXEOl2owViV9E5K1pJ/Luf.44w/Ci69ZBbFT6o...,1.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1186.0,$2a$15$aHTw2jbPTbRKGeDBfcQ25.KPs1kJF/UqcPukXAC...,1.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1194.0,$2a$15$CgRoc5cbNZ0QTWMIqNSbu..3w/a0GkYDa3ktG1x...,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1186.0,$2a$15$Ejn9P.vrIOxTGiqTGICiDeLirjfUfjv4xBdI4h6...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1202.0,YUqcVvVPHekQ3pk5dsZJb91EacfOFwSq9yrH4Gab7GbT8O...,1.0
1535,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1196.0,YX37GM6UxzX5a19UpwoQ6J4bq4x6Obi/zOIjqRmAfPWmBc...,1.0
1536,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1194.0,zeMsirklPKg0Ts2/jhhHTE4QTDubdJ6VAh420a2SxyZdWj...,1.0
1537,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1202.0,zmNzWWYP45nMMEl7vSBuaNnqWtUycLozveGSKlGoXanF8B...,1.0


In [8]:
features_cs110_testparams = getStudentFeatures('CS110', x1param = SUCCESS_GRADES, y1param = SUCCESS_GRADES)
features_cs110_testparams

,MATH51,CHEM31X,CS106X,CHEM33,PHIL1,CS103,CHEM35,CS109,CS161,CS107,...,PHYSICS230,PHYSICS231,BIO153,MATH220,PHYSICS62,ME105,AA272C,strm,emplid,y
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1202.0,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1192.0,$2a$15$5tXEOl2owViV9E5K1pJ/Luf.44w/Ci69ZBbFT6o...,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1186.0,$2a$15$aHTw2jbPTbRKGeDBfcQ25.KPs1kJF/UqcPukXAC...,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1194.0,$2a$15$CgRoc5cbNZ0QTWMIqNSbu..3w/a0GkYDa3ktG1x...,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1186.0,$2a$15$Ejn9P.vrIOxTGiqTGICiDeLirjfUfjv4xBdI4h6...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1202.0,YUqcVvVPHekQ3pk5dsZJb91EacfOFwSq9yrH4Gab7GbT8O...,1.0
1535,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1196.0,YX37GM6UxzX5a19UpwoQ6J4bq4x6Obi/zOIjqRmAfPWmBc...,1.0
1536,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1194.0,zeMsirklPKg0Ts2/jhhHTE4QTDubdJ6VAh420a2SxyZdWj...,1.0
1537,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1202.0,zmNzWWYP45nMMEl7vSBuaNnqWtUycLozveGSKlGoXanF8B...,1.0


In [9]:
full_cs110, prereqs_cs110 = getStudentFeatures("CS110", prereqs = True)

In [10]:
full_cs110

,MATH51,CHEM31X,CS106X,CHEM33,PHIL1,CS103,CHEM35,CS109,CS161,CS107,...,PHYSICS230,PHYSICS231,BIO153,MATH220,PHYSICS62,ME105,AA272C,strm,emplid,y
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1202.0,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1192.0,$2a$15$5tXEOl2owViV9E5K1pJ/Luf.44w/Ci69ZBbFT6o...,1.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1186.0,$2a$15$aHTw2jbPTbRKGeDBfcQ25.KPs1kJF/UqcPukXAC...,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1194.0,$2a$15$CgRoc5cbNZ0QTWMIqNSbu..3w/a0GkYDa3ktG1x...,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1186.0,$2a$15$Ejn9P.vrIOxTGiqTGICiDeLirjfUfjv4xBdI4h6...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1202.0,YUqcVvVPHekQ3pk5dsZJb91EacfOFwSq9yrH4Gab7GbT8O...,1.0
1535,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1196.0,YX37GM6UxzX5a19UpwoQ6J4bq4x6Obi/zOIjqRmAfPWmBc...,1.0
1536,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1194.0,zeMsirklPKg0Ts2/jhhHTE4QTDubdJ6VAh420a2SxyZdWj...,1.0
1537,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1202.0,zmNzWWYP45nMMEl7vSBuaNnqWtUycLozveGSKlGoXanF8B...,1.0


In [11]:
prereqs_cs110

,CS107,strm,emplid,y
0,1.0,1202.0,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,1.0
1,1.0,1192.0,$2a$15$5tXEOl2owViV9E5K1pJ/Luf.44w/Ci69ZBbFT6o...,1.0
2,1.0,1186.0,$2a$15$aHTw2jbPTbRKGeDBfcQ25.KPs1kJF/UqcPukXAC...,0.0
3,0.0,1194.0,$2a$15$CgRoc5cbNZ0QTWMIqNSbu..3w/a0GkYDa3ktG1x...,1.0
4,1.0,1186.0,$2a$15$Ejn9P.vrIOxTGiqTGICiDeLirjfUfjv4xBdI4h6...,0.0
...,...,...,...,...
1534,1.0,1202.0,YUqcVvVPHekQ3pk5dsZJb91EacfOFwSq9yrH4Gab7GbT8O...,1.0
1535,1.0,1196.0,YX37GM6UxzX5a19UpwoQ6J4bq4x6Obi/zOIjqRmAfPWmBc...,1.0
1536,1.0,1194.0,zeMsirklPKg0Ts2/jhhHTE4QTDubdJ6VAh420a2SxyZdWj...,1.0
1537,1.0,1202.0,zmNzWWYP45nMMEl7vSBuaNnqWtUycLozveGSKlGoXanF8B...,1.0


# Logistic regression models

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support

In [8]:
UNSEEN_QUARTERS = [1204]
TEST_QUARTERS = [1194, 1196] # aka "dev" quarters

def get_and_split_data_double_matrix(target_course, test_quarters = TEST_QUARTERS, gender = False, gpa = False, x1param = PASS_GRADES, y1param = SUCCESS_GRADES):
    dataset_full, dataset_prereq = getStudentFeatures(target_course, gender = gender, gpa = gpa, x1param = x1param, y1param = y1param, prereqs = True)
    
    # process full dataset
    
    dataset_grades_full = dataset_full.loc[~dataset_full['strm'].isin(UNSEEN_QUARTERS)] #For now, dropping this quarter
    dataset_train_full = dataset_grades_full.loc[~dataset_grades_full['strm'].isin(test_quarters)]
    dataset_test_full = dataset_grades_full.loc[dataset_grades_full['strm'].isin(test_quarters)]

    test_students_full = pd.DataFrame(list(dataset_test_full["emplid"]), columns = ["emplid"])
    if gender:
        test_students_full["female"] = list(dataset_test_full["female"])
    if gpa:
        test_students_full["gpa_A"] = list(dataset_test_full["gpa_A"])
        test_students_full["gpa_B"] = list(dataset_test_full["gpa_B"])
        test_students_full["gpa_C"] = list(dataset_test_full["gpa_C"])
        test_students_full["gpa_D"] = list(dataset_test_full["gpa_D"])
        test_students_full["gpa_NP"] = list(dataset_test_full["gpa_NP"])
        test_students_full["gpa_NaN"] = list(dataset_test_full["gpa_NaN"])
    dataset_train_full = dataset_train_full.drop(["strm", "emplid"], axis=1)
    dataset_test_full = dataset_test_full.drop(["strm", "emplid"], axis=1)

    X_train_full = dataset_train_full.drop(["y"], axis=1)
    y_train_full = dataset_train_full["y"]
    X_test_full = dataset_test_full.drop(["y"], axis=1)
    y_test_full = dataset_test_full["y"]
    
    if not len(X_train_full) or not len(X_test_full) or not len(y_train_full) or not len(y_test_full):
        return (None, None, None, None, None, True), (None, None, None, None, None, True)
    
    # process official prereq dataset
    
    dataset_grades_prereq = dataset_prereq.loc[~dataset_prereq['strm'].isin(UNSEEN_QUARTERS)] #For now, dropping this quarter
    dataset_train_prereq = dataset_grades_prereq.loc[~dataset_grades_prereq['strm'].isin(test_quarters)]
    dataset_test_prereq = dataset_grades_prereq.loc[dataset_grades_prereq['strm'].isin(test_quarters)]

    test_students_prereq = pd.DataFrame(list(dataset_test_prereq["emplid"]), columns = ["emplid"])
    if gender:
        test_students_prereq["female"] = list(dataset_test_prereq["female"])
    if gpa:
        test_students_prereq["gpa_A"] = list(dataset_test_prereq["gpa_A"])
        test_students_prereq["gpa_B"] = list(dataset_test_prereq["gpa_B"])
        test_students_prereq["gpa_C"] = list(dataset_test_prereq["gpa_C"])
        test_students_prereq["gpa_D"] = list(dataset_test_prereq["gpa_D"])
        test_students_prereq["gpa_NP"] = list(dataset_test_prereq["gpa_NP"])
        test_students_prereq["gpa_NaN"] = list(dataset_test_prereq["gpa_NaN"])
    dataset_train_prereq = dataset_train_prereq.drop(["strm", "emplid"], axis=1)
    dataset_test_prereq = dataset_test_prereq.drop(["strm", "emplid"], axis=1)

    X_train_prereq = dataset_train_prereq.drop(["y"], axis=1)
    y_train_prereq = dataset_train_prereq["y"]
    X_test_prereq = dataset_test_prereq.drop(["y"], axis=1)
    y_test_prereq = dataset_test_prereq["y"]
    
    if not len(X_train_prereq) or not len(X_test_prereq) or not len(y_train_prereq) or not len(y_test_prereq):
        return (None, None, None, None, None, True), (None, None, None, None, None, True)
    
    return [X_train_full, X_test_full, y_train_full, y_test_full, test_students_full, False], [X_train_prereq, X_test_prereq, y_train_prereq, y_test_prereq, test_students_prereq, False]

In [9]:
# Use the X predictor matrix and y outcome vector from 
# getStudentFeatures, fit a logistic regression model and return
# the list of coefficients, default sorted by absolute value

# Params: sort = ["pos", "neg", "abs"] to sort by largest positive, negative, or
# absolute value of coefficients
def get_coefs_from_split(X_train, X_test, y_train, y_test, fail_flag = False, sort = "abs"):
    if fail_flag:
        return None, None, None, None, None, None, None, None, None, True
    if len(y_train.unique()) == 1:
        return None, None, None, None, None, None, None, None, None, True
    model = LogisticRegression(solver = "lbfgs")
    model.fit(X_train, y_train)
    train_score = model.score(X_train, y_train) 
    test_score = model.score(X_test, y_test)
    train_groundtruth = y_train.mean()
    test_groundtruth = y_test.mean()
    train_size = len(y_train)
    test_size = len(y_test)
    y_predict = model.predict(X_test)
    prob_1_index = 1
    if model.classes_[1] != 1:
        prob_1_index = 0
    y_prob = [x[prob_1_index] for x in model.predict_proba(X_test)]
    scores = precision_recall_fscore_support(y_test, y_predict, average = "binary", zero_division = 0)
    coefs = [(X_test.columns[i], model.coef_[0][i]) for i in range(len(X_test.columns))]
    if sort:
        if sort == "pos":
            coefs = sorted(coefs, key = lambda x: x[1], reverse = True)
        if sort == "neg":
            coefs = sorted(coefs, key = lambda x: x[1], reverse = False)
        if sort == "abs":
            coefs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
    return coefs, y_predict, y_prob, train_score, test_score, train_groundtruth, test_groundtruth, train_size, test_size, scores, False

# Loop through all courses

In [12]:
from concurrent.futures import ThreadPoolExecutor

In [13]:
TEST_QUARTERS = [1194, 1196] # aka "dev" quarters

# with prereqs
def run_predictions_with_prereqs(course_name, results_full, results_prereq, test_quarters = TEST_QUARTERS, 
                                 gender = False, gpa = False, x1param = PASS_GRADES, y1param = SUCCESS_GRADES):
    [X_train_full, X_test_full, y_train_full, y_test_full, test_students_full, fail_flag_full], \
        [X_train_prereq, X_test_prereq, y_train_prereq, y_test_prereq, test_students_prereq, fail_flag_prereq] = \
        get_and_split_data_double_matrix(course_name, test_quarters = test_quarters, gender = gender, gpa = gpa, 
                                     x1param = x1param, y1param = y1param)
    
    print(fail_flag_full)
    print(fail_flag_prereq)
    # full results
    coefs, y_predict, y_prob, train_score, test_score, train_groundtruth, test_groundtruth, train_size, \
        test_size, scores, fail_flag = get_coefs_from_split(X_train_full, X_test_full, y_train_full, 
                                                            y_test_full, fail_flag_full)
    test_students_full[course_name + "_y"] = list(y_test_full)
    test_students_full[course_name + "_y_predict"] = y_predict
    test_students_full[course_name + "_y_prob"] = y_prob
    results_full.append([course_name, train_score, test_score, train_groundtruth, test_groundtruth, 
                         train_size, test_size, scores[0], scores[1], scores[2]])

    # official prereq results
    coefs, y_predict, y_prob, train_score, test_score, train_groundtruth, test_groundtruth, train_size, \
        test_size, scores, fail_flag = get_coefs_from_split(X_train_prereq, X_test_prereq, y_train_prereq, 
                                                            y_test_prereq, fail_flag_prereq)
    test_students_prereq[course_name + "_y"] = list(y_test_prereq)
    test_students_prereq[course_name + "_y_predict"] = y_predict
    test_students_prereq[course_name + "_y_prob"] = y_prob
    results_prereq.append([course_name, train_score, test_score, train_groundtruth, test_groundtruth, 
                           train_size, test_size, scores[0], scores[1], scores[2]])

In [14]:
results_full = []
results_prereq = []

run_predictions_with_prereqs("CS221", results_full, results_prereq)

print(results_full)
print(results_prereq)

False
False
[['CS221', 0.8021978021978022, 0.7151162790697675, 0.7009419152276295, 0.7906976744186046, 1274, 172, 0.8, 0.8529411764705882, 0.8256227758007119]]
[['CS221', 0.7009419152276295, 0.7906976744186046, 0.7009419152276295, 0.7906976744186046, 1274, 172, 0.7906976744186046, 1.0, 0.8831168831168831]]


In [19]:
import sklearn
print(sklearn.__version__)

0.19.1


### x1 success y1 success no gpa no gender

In [24]:
results_full = []
results_prereq = []
executor = ThreadPoolExecutor()
for course_name in all_courses:
    executor.submit(run_predictions_with_prereqs, course_name, results_full, results_prereq, TEST_QUARTERS, 
                                 False, False, SUCCESS_GRADES, SUCCESS_GRADES)
executor.shutdown()
results_full_df = pd.DataFrame(results_full, columns = ["course_name", "train_score", "test_score", "train_groundtruth", "test_groundtruth", "train_size", "test_size", "precision", "recall", "f1_score"])
results_prereq_df = pd.DataFrame(results_prereq, columns = ["course_name", "train_score", "test_score", "train_groundtruth", "test_groundtruth", "train_size", "test_size", "precision", "recall", "f1_score"])

*****FULL*****
Index(['PHYSICS61', 'PHYSICS63', 'PHYSICS65', 'CME302', 'MATH220', 'CME305',
       'EE124', 'CME306', 'CS221', 'PHIL80', 'PHIL102', 'PHIL150', 'EE261',
       'STATS191', 'strm', 'emplid', 'y'],
      dtype='object')
*****PREREQ*****
Index(['CME302', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CS106X', 'CME100', 'CS103', 'CME102', 'CS107', 'PHYSICS41', 'CME104',
       'PHYSICS43', 'CS110', 'ENGR40', 'CS144', 'PHYSICS45', 'EE101A',
       'EE102A', 'CS140', 'EE101B', 'CS109', 'EE102B', 'MATH104', 'ENGR105',
       'ENGR205', 'EE263', 'EE271', 'EE122A', 'EE180', 'ENGR62', 'EE109',
       'EE214A', 'ENGR209A', 'EE133', 'strm', 'emplid', 'y'],
      dtype='object')
*****PREREQ*****
Index(['ENGR205', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
*****FULL*****Index(['MATH42', 'PHYSICS41', 'MATH51', 'PHYSICS43', 'CME102', 'PHYSICS45',
       'ENERGY101', 'ENGR14', 'CHEM31B', 'CHEM33', 'CEE177', 'CEE101A',
       'CEE171', 'CEE160', 'CEE161A', 'CEE

*****FULL*****
Index(['CHEM31A', 'CHEM31B', 'MATH51', 'PSYCH1', 'CHEM33', 'MATH53', 'ENGR14',
       'BIO41', 'PHYSICS41', 'ENGR30', 'ENGR15', 'CME106', 'BIO42', 'ME70',
       'ME101', 'AA240A', 'ME80', 'ME112', 'ME203', 'MATH42', 'ECON1',
       'PHYSICS43', 'PHYSICS45', 'CS106A', 'STATS110', 'ME131A', 'AA100',
       'ME131B', 'ME261', 'AA240B', 'ME140', 'strm', 'emplid', 'y'],
      dtype='object')
*****PREREQ*****
Index(['ENGR14', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['MATH42', 'PHYSICS41', 'MATH51', 'PHYSICS43', 'CME102', 'PHYSICS45',
       'ENERGY101', 'ENGR14', 'CHEM31B', 'CHEM33', 'CEE177', 'CEE101A',
       'CEE171', 'CEE160', 'CEE161A', 'CEE101B', 'CEE172', 'CEE166A',
       'CEE101C', 'STATS60', 'CEE166B', 'ENGR30', 'CEE169', 'CME100',
       'CHEM31A', 'CME102A', 'CEE180', 'CEE182', 'CS106A', 'CEE181', 'CEE183',
       'CS106X', 'CS107', 'CEE176B', 'CS103', 'MATH51M', 'CS109', 'CS161',
       'CS110', 'ENGR40M', 'CS145', 'CS221', 'MATH19', 'MATH20'

*****FULL*****
Index(['CME100', 'CS106B', 'PHYSICS41', 'PHYSICS43', 'ENGR40M', 'CS107',
       'CS108', 'CS103', 'ME101', 'CS109', 'CME103', 'CS161', 'CS124', 'CS110',
       'CS221', 'CS210A', 'BIOMEDIN371', 'CS210B', 'CS231N', 'CS224U', 'CS154',
       'CS228', 'CS168', 'MATH51', 'SYMSYS100', 'CHEM31X', 'PHIL150',
       'PSYCH60B', 'CHEM33', 'CHEM35', 'PHIL180', 'BIO41', 'PHYSICS21',
       'PHYSICS23', 'PHYSICS25', 'LINGUIST1', 'PHIL80', 'PSYCH50', 'PSYCH1',
       'CHEM31A', 'CHEM31B', 'CS106A', 'STATS141', 'BIO42', 'CS131', 'CS274',
       'CS279', 'PHYSICS45', 'BIOMEDIN217', 'ENGR14', 'strm', 'emplid', 'y'],
      dtype='object')
*****PREREQ*****
Index(['CS106A', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CHEM31A', 'MATH41', 'MATH42', 'CHEM31B', 'PHYSICS41', 'PHYSICS43',
       'CME100', 'PHYSICS45', 'GEOPHYS110', 'EARTHSYS101', 'EARTHSYS164',
       'EARTHSYS112', 'EARTHSYS111', 'EARTHSYS151', 'EARTHSYS146B',
       'EARTHSYS146A', 'BIO144', 'EARTHSYS291', '

*****FULL*****
Index(['CME102', 'PHYSICS41', 'PHYSICS43', 'CME100', 'ECON102A', 'ECON102B',
       'ECON137', 'ECON136', 'ECON50', 'ECON52',
       ...
       'GEOPHYS110', 'PHIL150', 'CS102', 'BIOE42', 'BIOE101', 'STATS271',
       'CS154', 'strm', 'emplid', 'y'],
      dtype='object', length=141)
*****PREREQ*****
Index(['ECON50', 'ECON51', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CS106A', 'MATH21', 'ENGR40M', 'ME101', 'CS106B', 'CME100', 'PSYCH1',
       'PHYSICS42', 'PHYSICS41', 'PHYSICS43', 'ME115A', 'ENGR14', 'ME80',
       'ME115B', 'STATS60', 'MS&E120', 'ME216A', 'CHEM31A', 'CME100A',
       'ENERGY101', 'CHEM31B', 'MATH42', 'PHYSICS45', 'CME102', 'ENGR30',
       'ENGR15', 'ENGR40', 'ME70', 'ECON1', 'ME131A', 'ME112', 'ME203',
       'ME105', 'ME161', 'AA100', 'ME131B', 'ME113', 'CHEM31X', 'CHEM33',
       'MATH51', 'CHEM35', 'CHEM131', 'CHEM151', 'CS107', 'CHEM130', 'CS103',
       'PHIL1', 'MATH19', 'MATH20', 'CME106', 'BIO104', 'CS109', 'PHYSICS41A',
  

Index(['ME351A', 'strm', 'emplid', 'y'], dtype='object')
*****FULL**********FULL*****

Index(['CS106X', 'MATH51', 'CS107', 'CS193A', 'ENGR62', 'CS109', 'CS103',
       'CS148', 'CS110', 'CS221',
       ...
       'MATH107', 'PSYCH164', 'CS224U', 'MS&E226', 'EE109', 'MATH172',
       'MATH215A', 'strm', 'emplid', 'y'],
      dtype='object', length=215)
*****PREREQ*****
Index(['CS107', 'strm', 'emplid', 'y'], dtype='object')
Index(['PHYSICS61', 'CS106A', 'CS106B', 'PHYSICS43', 'CS103', 'PHYSICS45',
       'PHYSICS70', 'CS107', 'CS109', 'GS4',
       ...
       'PHYSICS91SI', 'MATH121', 'BIOE210', 'MATSCI203', 'MATH19', 'MATH20',
       'STATS110', 'strm', 'emplid', 'y'],
      dtype='object', length=315)
*****PREREQ*****
Index(['PHYSICS41', 'PHYSICS43', 'MATH53', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CS106A', 'CHEM31A', 'MATH41', 'CHEM31B', 'PHYSICS41', 'MATH42',
       'PHYSICS43', 'LINGUIST1', 'CME100', 'AA100', 'CS106B', 'PHYSICS45',
       'CME102', 'EE101A',

...

*****FULL*****
Index(['CHEM31A', 'CHEM31B', 'MATH51', 'CS106A', 'CHEM33', 'CS106B', 'BIO41',
       'CS103', 'BIO42', 'PHYSICS41A',
       ...
       'CME102A', 'CS103A', 'BIOMEDIN224', 'PSYCH90', 'BIO153', 'RAD260',
       'CHEM171', 'strm', 'emplid', 'y'],
      dtype='object', length=101)
*****PREREQ*****
Index(['CS106A', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CS106A', 'CS106B', 'MATH51', 'CS107', 'ECON1', 'CS110', 'CS147',
       'CS142', 'ECON102A', 'ECON50',
       ...
       'STATS110', 'PSYCH90', 'ENGR30', 'ME70', 'ECON157', 'ECON141', 'ME105',
       'strm', 'emplid', 'y'],
      dtype='object', length=198)
*****PREREQ*****
Index(['CS147', 'CS148', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['MATH51', 'SYMSYS100', 'CS106B', 'CS142', 'ENGR40M', 'CS107', 'MATH104',
       'CS103', 'CS124', 'CS109',
       ...
       'BIOE42', 'BIOE101', 'STATS271', 'ECON159', 'CS276', 'MATH220', 'EE42',
       'strm', 'emplid', 'y'],
      dtype='object'

*****FULL*****
Index(['ECON1', 'MATH20', 'MATH21', 'MATH51', 'PHIL150', 'CS106A', 'PSYCH50',
       'CS106B', 'PHIL80', 'CS131',
       ...
       'BIOE101', 'CS107E', 'CME103', 'CS242', 'CS193A', 'MS&E252',
       'LINGUIST130B', 'strm', 'emplid', 'y'],
      dtype='object', length=193)
*****PREREQ*****
Index(['PSYCH30', 'PSYCH50', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CME100', 'ENGR14', 'CS106X', 'ENGR40M', 'AA241X', 'CS107E', 'CS103',
       'AA100', 'ENGR15', 'ENGR30',
       ...
       'MATH51M', 'EE122A', 'LINGUIST1', 'EE122B', 'MATH104', 'ENGR62',
       'MATSCI152', 'strm', 'emplid', 'y'],
      dtype='object', length=141)
*****PREREQ*****
Index(['ENGR105', 'MATH113', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['MATH51', 'CS106A', 'ECON50', 'HUMBIO2A', 'HUMBIO3A', 'ECON102A',
       'HUMBIO3B', 'ECON52', 'ECON102B', 'ECON137',
       ...
       'CS142', 'CS124', 'CS247', 'CS157', 'CS376', 'CS210A', 'COMM273D',
       'strm', 'emplid', 

*****FULL*****
Index(['CS106A', 'CME100A', 'CME102A', 'PHYSICS41A', 'PHYSICS41', 'ENGR14',
       'PHYSICS43', 'ENGR40M', 'EE108', 'CS106B',
       ...
       'CS241', 'ME112', 'EE252', 'PSYCH131', 'EE42', 'MATH113', 'EE254',
       'strm', 'emplid', 'y'],
      dtype='object', length=142)
*****PREREQ*****
Index(['ENGR40', 'EE122A', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CHEM31A', 'CHEM31B', 'MATH51', 'CHEM33', 'HUMBIO2A', 'HUMBIO3A',
       'HUMBIO3B', 'HUMBIO4A', 'PSYCH50', 'PSYCH80',
       ...
       'STATS240', 'ENERGY102', 'CEE176B', 'CS294S', 'CS193W', 'PHYSICS70',
       'PHIL359', 'strm', 'emplid', 'y'],
      dtype='object', length=352)
*****PREREQ*****
Index(['PSYCH1', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['MATH41', 'CHEM31X', 'MATH42', 'CHEM33', 'BIO41', 'CHEM35', 'BIO42',
       'BIO43', 'MI185', 'BIO158', 'PHYSICS25', 'CHEM31A', 'MATH20', 'CHEM31B',
       'MATH21', 'PHYSICS41A', 'ENGR14', 'PHYSICS23', 'HUMBIO129S', 'HUMBIO2

*****FULL*****
Index(['CHEM31A', 'CHEM31B', 'MATH51', 'CHEM33', 'HUMBIO2A', 'HUMBIO3A',
       'HUMBIO3B', 'HUMBIO4A', 'PSYCH50', 'PSYCH80',
       ...
       'EARTHSYS112', 'STATS50', 'ENERGY101', 'STATS202', 'CME108', 'MS&E221',
       'PHIL186', 'strm', 'emplid', 'y'],
      dtype='object', length=167)
*****PREREQ*****
Index(['PSYCH60', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CS106A', 'CME100', 'CS106B', 'CME102', 'ENGR40M', 'CS107', 'CS148',
       'EE103', 'EE108', 'EE180',
       ...
       'CS107E', 'CS316', 'CME104', 'CS229', 'STATS202', 'STATS200', 'EE216',
       'strm', 'emplid', 'y'],
      dtype='object', length=110)
*****PREREQ*****
Index(['EE101A', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
*****FULL*****
Index(['MATH51H', 'PHYSICS61', 'PHYSICS63', 'MATH53', 'CS106B', 'PHYSICS65',
       'CS103', 'MATH171', 'EE101A', 'PSYCH1',
       ...
       'STATS202', 'PHYSICS113', 'CME104', 'EE155', 'PHYSICS152', 'EE293B',
       'ECON1', 'strm', 

*****FULL*****
Index(['MATH51', 'CHEM31X', 'CS106X', 'CHEM33', 'PHIL1', 'CS103', 'CHEM35',
       'CS109', 'CS161', 'CS107',
       ...
       'PHYSICS230', 'PHYSICS231', 'BIO153', 'MATH220', 'PHYSICS62', 'ME105',
       'AA272C', 'strm', 'emplid', 'y'],
      dtype='object', length=539)
*****PREREQ*****
Index(['CS107', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CHEM31A', 'ECON1', 'CS106A', 'CME100A', 'CHEM31X', 'CME102',
       'PHYSICS41', 'ENGR14', 'ME80', 'PHYSICS45',
       ...
       'ECON179', 'MATH137', 'PHIL187', 'MATH148', 'MATH205A', 'ESS258',
       'CHEMENG274', 'strm', 'emplid', 'y'],
      dtype='object', length=555)
*****PREREQ*****
Index(['PHYSICS41', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CS106A', 'CS106B', 'MATH51', 'CS107', 'CS109', 'ME101', 'CS124',
       'CS103', 'CS221', 'PHYSICS43',
       ...
       'EE216', 'PHYSICS62', 'PHYSICS91SI', 'LINGUIST180', 'LINGUIST222A',
       'LINGUIST230B', 'MATH151', 'strm', 'emplid',

In [25]:
results_full_df

,course_name,train_score,test_score,train_groundtruth,test_groundtruth,train_size,test_size,precision,recall,f1_score
0,EARTHSYS158,1.000000,0.500000,0.714286,0.500000,7,2,0.500000,1.000000,0.666667
1,EARTHSYS151,0.954545,1.000000,0.954545,1.000000,22,4,1.000000,1.000000,1.000000
2,HUMBIO136,0.941176,1.000000,0.941176,1.000000,17,1,1.000000,1.000000,1.000000
3,CS275,1.000000,1.000000,0.800000,1.000000,25,5,1.000000,1.000000,1.000000
4,BIO115,0.888889,1.000000,0.777778,1.000000,27,15,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...
268,CS221,0.840659,0.773256,0.700942,0.790698,1274,172,0.864662,0.845588,0.855019
269,PSYCH1,0.764335,0.793548,0.755918,0.796774,1901,310,0.801980,0.983806,0.883636
270,CS106B,0.737635,0.687140,0.694399,0.662188,3053,521,0.696121,0.936232,0.798517
271,CS161,0.814843,0.763077,0.570465,0.670769,1334,325,0.830986,0.811927,0.821346


In [26]:
results_prereq_df

,course_name,train_score,test_score,train_groundtruth,test_groundtruth,train_size,test_size,precision,recall,f1_score
0,EARTHSYS151,0.954545,1.000000,0.954545,1.000000,22,4,1.000000,1.000000,1.000000
1,EARTHSYS158,0.714286,0.500000,0.714286,0.500000,7,2,0.500000,1.000000,0.666667
2,HUMBIO136,0.941176,1.000000,0.941176,1.000000,17,1,1.000000,1.000000,1.000000
3,BIO115,0.777778,1.000000,0.777778,1.000000,27,15,1.000000,1.000000,1.000000
4,CS275,0.800000,1.000000,0.800000,1.000000,25,5,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...
228,CS109,0.724156,0.753846,0.633247,0.810256,1925,390,0.940000,0.743671,0.830389
229,PHYSICS41,0.666186,0.646809,0.664744,0.646809,1387,470,0.646809,1.000000,0.785530
230,CS221,0.773940,0.825581,0.700942,0.790698,1274,172,0.907692,0.867647,0.887218
231,CS106B,0.694399,0.662188,0.694399,0.662188,3053,521,0.662188,1.000000,0.796767


In [28]:
RESULTS_FULL_PATH = "../results/nogender_nogpa_x1success_y1success_full.fthr"
RESULTS_PREREQ_PATH = "../results/nogender_nogpa_x1success_y1success_prereq.fthr"
results_full_df.to_feather(RESULTS_FULL_PATH)
results_prereq_df.to_feather(RESULTS_PREREQ_PATH)

### x1 pass y1 pass no gpa no gender

In [29]:
results_full = []
results_prereq = []
executor = ThreadPoolExecutor()
for course_name in all_courses:
    executor.submit(run_predictions_with_prereqs, course_name, results_full, results_prereq, TEST_QUARTERS, 
                                 False, False, PASS_GRADES, PASS_GRADES)
executor.shutdown()
results_full_df = pd.DataFrame(results_full, columns = ["course_name", "train_score", "test_score", "train_groundtruth", "test_groundtruth", "train_size", "test_size", "precision", "recall", "f1_score"])
results_prereq_df = pd.DataFrame(results_prereq, columns = ["course_name", "train_score", "test_score", "train_groundtruth", "test_groundtruth", "train_size", "test_size", "precision", "recall", "f1_score"])

*****FULL*****
Index(['PHYSICS61', 'PHYSICS63', 'PHYSICS65', 'CME302', 'MATH220', 'CME305',
       'EE124', 'CME306', 'CS221', 'PHIL80', 'PHIL102', 'PHIL150', 'EE261',
       'STATS191', 'strm', 'emplid', 'y'],
      dtype='object')
*****PREREQ*****
Index(['CME302', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CS106X', 'CME100', 'CS103', 'CME102', 'CS107', 'PHYSICS41', 'CME104',
       'PHYSICS43', 'CS110', 'ENGR40', 'CS144', 'PHYSICS45', 'EE101A',
       'EE102A', 'CS140', 'EE101B', 'CS109', 'EE102B', 'MATH104', 'ENGR105',
       'ENGR205', 'EE263', 'EE271', 'EE122A', 'EE180', 'ENGR62', 'EE109',
       'EE214A', 'ENGR209A', 'EE133', 'strm', 'emplid', 'y'],
      dtype='object')
*****PREREQ*****
Index(['ENGR205', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CME100', 'PHYSICS61', 'PHYSICS63', 'CME102', 'CS106A', 'CME104',
       'ENGR30', 'ME70', 'ME101', 'ENGR14', 'ME131A', 'ENGR15', 'ME203',
       'CME106', 'ME131B', 'ME204A', 'ME80', 'ECON1', 'ME2

*****FULL*****
Index(['CHEM31A', 'CHEM31B', 'MATH51', 'PSYCH1', 'CHEM33', 'MATH53', 'ENGR14',
       'BIO41', 'PHYSICS41', 'ENGR30', 'ENGR15', 'CME106', 'BIO42', 'ME70',
       'ME101', 'AA240A', 'ME80', 'ME112', 'ME203', 'MATH42', 'ECON1',
       'PHYSICS43', 'PHYSICS45', 'CS106A', 'STATS110', 'ME131A', 'AA100',
       'ME131B', 'ME261', 'AA240B', 'ME140', 'strm', 'emplid', 'y'],
      dtype='object')
*****PREREQ*****
Index(['ENGR14', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CME100', 'CHEM31X', 'CME102', 'GEOPHYS120', 'GEOPHYS110', 'PHYSICS43',
       'GS102', 'PHYSICS45', 'ME80', 'GS103', 'GS111', 'GEOPHYS162', 'GS105',
       'GS171', 'GS4', 'GS90', 'GS190', 'GS180', 'CME104', 'GS104', 'ESS220',
       'GEOPHYS222', 'ENERGY160', 'GS185', 'GEOPHYS183', 'PHYSICS113',
       'MATH41', 'CHEM31A', 'ENERGY101', 'CHEM31B', 'MATH20', 'CHEM33',
       'MATH21', 'PHYSICS41', 'PHYSICS41A', 'GS151', 'GS110', 'PHYSICS21',
       'PHYSICS22', 'CS106A', 'EARTHSYS158', 'GS253'

*****FULL*****
Index(['CS106X', 'CS107', 'EE178', 'EE65', 'CHEM33', 'CS229', 'CS161',
       'PHIL80', 'EE101A', 'EE102A', 'EE364A', 'EE102B', 'EE278', 'EE108',
       'MATH205A', 'EE376A', 'CME309', 'CS221', 'MATH220', 'strm', 'emplid',
       'y'],
      dtype='object')
*****PREREQ*****
Index(['CS161', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
*****FULL*****
Index(['CME100', 'CS106B', 'CME102', 'PHYSICS41', 'CS107E', 'CS103',
       'PHYSICS43', 'STATS116', 'MATH113', 'CS124',
       ...
       'PHIL170', 'ME131B', 'ME140', 'ME131A', 'AA210A', 'PHYSICS45', 'ME227',
       'strm', 'emplid', 'y'],
      dtype='object', length=101)
*****PREREQ*****
Index(['strm', 'emplid', 'y'], dtype='object')
Index(['AA100', 'CME100', 'CME102', 'PHYSICS41', 'PHYSICS43', 'STATS110',
       'PHYSICS45', 'ENGR30', 'CS106A', 'ENGR14', 'ENGR15', 'ME70', 'ME131A',
       'ME161', 'ME80', 'ENGR105', 'ME227', 'AA256', 'AA242A', 'CHEM31X',
       'AA241A', 'AA241B', 'CEE70', 'CS106X', 'CS106B', 'C

      dtype='object')
*****PREREQ*****
Index(['EE376A', 'STATS300A', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CHEM31X', 'MATH51', 'CHEM33', 'CHEM35', 'BIO41', 'CHEM131', 'CHEM130',
       'BIO42', 'BIO43', 'EARTHSYS111', 'PHYSICS41', 'CHEM135', 'PHYSICS43',
       'ECON1', 'BIO144', 'CEE274A', 'BIO188', 'EARTHSYS112', 'BIO101',
       'CS106A', 'BIO189', 'PHYSICS45', 'MATH52', 'STATS191', 'CS106B',
       'STATS116', 'CEE274B', 'STATS202', 'MATH109', 'CHEMENG183', 'MS&E211',
       'MATH53', 'CS107', 'CS103', 'CS109', 'PHYSICS21', 'PSYCH1', 'PHYSICS23',
       'BIO104', 'PHYSICS25', 'CS157', 'CEE177', 'CS108', 'CME100', 'CEE164',
       'GS4', 'CEE166A', 'CEE101B', 'EARTHSYS151', 'ENGR14', 'CEE70', 'BIO158',
       'STATS60', 'BIO156', 'PSYCH95', 'CHEM31A', 'CME102A', 'CHEM31B',
       'CME104', 'ENGR30', 'ME101', 'ENERGY101', 'CME106', 'OSPPARIS40M',
       'ME70', 'CME108', 'EARTHSYS158', 'ME203', 'ENGR105', 'ME161', 'ME131A',
       'ME80', 'ME318', 'ME112', 'M

*****FULL*****
Index(['CME100', 'PHYSICS45', 'CS106A', 'CME102', 'PHYSICS41', 'ENGR14',
       'PHYSICS43', 'ENGR15', 'ENGR40', 'ENGR30', 'CME106', 'ME101', 'ME70',
       'ME131A', 'ME80', 'ME203', 'ME131B', 'ME140', 'ME161', 'ME351A',
       'ME112', 'ME351B', 'MATH148', 'MATH226', 'ME250', 'ENGR40M', 'ME300A',
       'CS181', 'ME113', 'ME300C', 'CS106B', 'CHEM31X', 'CEE70', 'MATH51',
       'ME210', 'MATH53', 'ME262', 'ME352C', 'strm', 'emplid', 'y'],
      dtype='object')
*****PREREQ*****
Index(['ME351A', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CHEM31X', 'CS106A', 'CME100', 'CME102', 'CHEM33', 'ECON1', 'BIO41',
       'ECON102A', 'BIOE41', 'ECON102B',
       ...
       'ME70', 'AA241B', 'PHIL170', 'ME161', 'ME131A', 'ENGR240', 'CS110',
       'strm', 'emplid', 'y'],
      dtype='object', length=154)
*****PREREQ*****
Index(['CHEM33', 'CHEM171', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CS106B', 'MATH51H', 'CS107', 'CS103', 'CS110', 'MATH12

...

Index(['strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CS229', 'CS265', 'MATH120', 'MATH121', 'MATH145', 'EE261', 'CS274',
       'EE364A', 'CS229T', 'CS231N', 'MATH51', 'BIOE211', 'STATS116',
       'MATH116', 'MATH147', 'MATH230A', 'MATH205A', 'STATS300A', 'MATH205B',
       'MATH230B', 'MATH230C', 'MATH122', 'PHIL170', 'MATH154', 'MATH210A',
       'MATH216A', 'CS228', 'strm', 'emplid', 'y'],
      dtype='object')
*****PREREQ*****
Index(['MATH230B', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CHEM31A', 'CHEM31B', 'MATH51', 'CS106A', 'CHEM33', 'CS106B', 'BIO41',
       'CS103', 'BIO42', 'PHYSICS41A',
       ...
       'CME102A', 'CS103A', 'BIOMEDIN224', 'PSYCH90', 'BIO153', 'RAD260',
       'CHEM171', 'strm', 'emplid', 'y'],
      dtype='object', length=101)
*****PREREQ*****
Index(['CS106A', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CS106A', 'CS106B', 'MATH51', 'CS107', 'ECON1', 'CS110', 'CS147',
       'CS142', 'ECON102A', 'ECON

*****PREREQ*****
Index(['CS248', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['MATH51H', 'PHYSICS61', 'PHYSICS63', 'MATH52H', 'CS106X', 'CS109',
       'MATH53H', 'PHYSICS65', 'CS107', 'MATH120', 'MATH171', 'PHYSICS112',
       'PHYSICS120', 'MATH131P', 'PHYSICS130', 'PHYSICS121', 'PHYSICS131',
       'PHYSICS134', 'PHYSICS110', 'PHYSICS105', 'PHYSICS107', 'MATH114',
       'CS161', 'PHYSICS161', 'PHYSICS152', 'PHIL1', 'CS142', 'CS108',
       'PHYSICS170', 'PHYSICS262', 'PHYSICS330', 'PHYSICS171', 'PHYSICS160',
       'PHYSICS108', 'MATH122', 'PHYSICS172', 'PHYSICS373', 'CS106A',
       'STATS116', 'STATS200', 'CS103', 'MS&E211', 'CS229', 'EE364A',
       'MATH173', 'MATH116', 'CS221', 'STATS219', 'CME108', 'MS&E221',
       'MATH236', 'PHYSICS372', 'PHYSICS212', 'MATH148', 'PHYSICS331',
       'MATH215B', 'PHYSICS351', 'PHYSICS332', 'MATH145', 'PHYSICS113',
       'CHEM31X', 'CHEM33', 'CS106B', 'PHYSICS211', 'MATH172', 'EE278',
       'CS238', 'APPPHYS273', 'PHIL170'

*****PREREQ*****
Index(['strm', 'emplid', 'y'], dtype='object')*****FULL*****

Index(['CHEM31A', 'CHEM31B', 'MATH51', 'CHEM33', 'HUMBIO2A', 'HUMBIO3A',
       'HUMBIO3B', 'HUMBIO4A', 'PSYCH50', 'PSYCH80',
       ...
       'STATS240', 'ENERGY102', 'CEE176B', 'CS294S', 'CS193W', 'PHYSICS70',
       'PHIL359', 'strm', 'emplid', 'y'],
      dtype='object', length=352)
*****PREREQ*****
Index(['PSYCH1', 'strm', 'emplid', 'y'], dtype='object')
*****FULL**********FULL*****
Index(['CME100', 'CHEM31X', 'CME102', 'CHEM33', 'CHEMENG20', 'CHEM35',
       'CHEMENG100', 'CME106', 'CHEMENG120A', 'CME104', 'CHEMENG120B',
       'CHEM171', 'CME211', 'CHEM131', 'CHEM173', 'CHEMENG110', 'CME108',
       'CHEM175', 'CHEMENG142', 'CHEMENG185A', 'CHEMENG181', 'CHEMENG174',
       'CS106A', 'MATH19', 'PHYSICS41', 'MATH20', 'PHYSICS41A', 'CS106B',
       'PHYSICS43', 'MATH21', 'ENGR14', 'CEE70', 'CEE160', 'STATS60',
       'CEE101B', 'PHYSICS45', 'ENGR50', 'MATH51', 'MATH52', 'MATSCI151',
       'PHYSICS42', 

*****FULL**********FULL*****
Index(['CS106X', 'CS107', 'MATH108', 'PHIL1', 'CS110', 'MATH107', 'CS103',
       'CS109', 'PHIL150', 'CS161',
       ...
       'PSYCH95', 'MS&E220', 'CME103', 'PHYSICS161', 'PHYSICS62', 'HUMBIO2A',
       'PSYCH141', 'strm', 'emplid', 'y'],
      dtype='object', length=328)
*****PREREQ*****
Index(['MATH51', 'strm', 'emplid', 'y'], dtype='object')

Index(['CS106B', 'MATH51H', 'CS103', 'CS107', 'CS110', 'MATH53H', 'CS161',
       'MATH171', 'CS109', 'CS154',
       ...
       'MATH151', 'CS107E', 'EE103', 'CS145', 'CS242', 'CS157', 'STATS240',
       'strm', 'emplid', 'y'],
      dtype='object', length=140)
*****PREREQ*****
Index(['MATH171', 'strm', 'emplid', 'y'], dtype='object')
*****FULL**********FULL*****
Index(['CS107', 'CS103', 'MATH51', 'CS109', 'CS228', 'CS161', 'CS110',
       'CS205A', 'CS261', 'CS148', 'CS242', 'MATH52', 'CS140', 'CS448Z',
       'CS243', 'PHYSICS63', 'MATH108', 'MATH171', 'CS348B', 'CME102', 'EE261',
       'MATH121', 'PHYSICS13

Index(['MATH20', 'MATH41', 'strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CS106A', 'CS106B', 'MATH51', 'CS107', 'CS109', 'CS103', 'CS110',
       'CS124', 'CS161', 'MATH104',
       ...
       'MS&E319', 'STATS231', 'EE271', 'CS377U', 'BIOE210', 'MATH137',
       'EARTHSYS112', 'strm', 'emplid', 'y'],
      dtype='object', length=541)
*****PREREQ*****
Index(['strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['CHEM31A', 'ECON1', 'MATH51', 'ECON143', 'CS106A', 'CS106B', 'CS109',
       'MATH21', 'CHEM31B', 'CME100',
       ...
       'CHEM33C', 'PHYSICS161', 'PHYSICS262', 'CME192', 'MATSCI193', 'ME105',
       'CEE274A', 'strm', 'emplid', 'y'],
      dtype='object', length=461)
*****PREREQ*****
Index(['strm', 'emplid', 'y'], dtype='object')
*****FULL*****
Index(['MATH51', 'CHEM31X', 'CS106X', 'CHEM33', 'PHIL1', 'CS103', 'CHEM35',
       'CS109', 'CS161', 'CS107',
       ...
       'CS193A', 'LINGUIST130B', 'MATSCI320', 'ESS258', 'CHEMENG274',
       'MATH220', 

In [30]:
RESULTS_FULL_PATH = "../results/nogender_nogpa_x1pass_y1pass_full.fthr"
RESULTS_PREREQ_PATH = "../results/nogender_nogpa_x1pass_y1pass_prereq.fthr"
results_full_df.to_feather(RESULTS_FULL_PATH)
results_prereq_df.to_feather(RESULTS_PREREQ_PATH)

## Iterate on log reg models inc. official prereqs

In [13]:
LETTER_GRADES = ["A+", "A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D+", "D", "D-", "NP", "W"]
PASS_GRADES = ["A+", "A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D+", "D", "D-"]
SUCCESS_GRADES = ["A+", "A", "A-", "B+"]
BMINUS_GRADES = ["A+", "A", "A-", "B+", "B", "B-"]

all_courses = set(raw_data["course_name"]) 

############# MODIFY THIS SECTION #############
courselist = all_courses

gender_param = True
gpa_param = False # currently crashes if set to True

x1grades_param = SUCCESS_GRADES
x1grades_shortname = "success" # short form for the filename
y1grades_param = SUCCESS_GRADES
y1grades_shortname = "success" # short form for the filename
limited_run = True
limit_count = 10
###############################################

results_full = []
results_prereq = []
executor = ThreadPoolExecutor()
count = 0
for course_name in courselist:
    executor.submit(run_predictions_with_prereqs, course_name, results_full, results_prereq, TEST_QUARTERS, 
                                 gender_param, gpa_param, x1grades_param, y1grades_param)
    if limited_run:
        print(course_name)
        count += 1
        if count == limit_count:
            break
executor.shutdown()
results_full_df = pd.DataFrame(results_full, columns = ["course_name", "train_score", "test_score", "train_groundtruth", "test_groundtruth", "train_size", "test_size", "precision", "recall", "f1_score"])
results_prereq_df = pd.DataFrame(results_prereq, columns = ["course_name", "train_score", "test_score", "train_groundtruth", "test_groundtruth", "train_size", "test_size", "precision", "recall", "f1_score"])

pprint(results_full_df)
pprint(results_prereq_df)

PATHNAME = "gender" + str(gender_param) + "_gpa" + str(gpa_param) + "_x1" + x1grades_shortname + "_y1" + y1grades_shortname
RESULTS_FULL_PATH = "../results/" + PATHNAME + "_full.fthr"
RESULTS_PREREQ_PATH = "../results/" + PATHNAME + "_prereq.fthr"
results_full_df.to_feather(RESULTS_FULL_PATH)
results_prereq_df.to_feather(RESULTS_PREREQ_PATH)

CME309
EE101B
MATH51M
MATH210A
ECON107
CHEMENG185B
CEE161A
PHYSICS44
CHEM271
BIOE141A
Empty DataFrame
Columns: [course_name, train_score, test_score, train_groundtruth, test_groundtruth, train_size, test_size, precision, recall, f1_score]
Index: []
Empty DataFrame
Columns: [course_name, train_score, test_score, train_groundtruth, test_groundtruth, train_size, test_size, precision, recall, f1_score]
Index: []


ValueError: feather does not support serializing <class 'pandas.core.indexes.base.Index'> for the index; you can .reset_index() to make the index into column(s)

In [15]:
pd.__version__

'1.0.1'

## LASSO logistic regression to reduce nonzero coefficients

In [ ]:
# TEST_QUARTERS = [1194, 1196] # aka "dev" quarters

# def one_course_lasso(course_name, test_quarters = TEST_QUARTERS, gender = False, gpa= False, prereqSuccess = False):
#     X_train, X_test, y_train, y_test, fail_flag = get_and_split_data(course_name, test_quarters, gender, gpa = gpa, prereqSuccess = prereqSuccess)
#     for i in [10, 3, 1, 0.3, 0.1, 0.03, 0.01]:
#         model = LogisticRegression(penalty="l1", solver="saga", C=i, max_iter=1000) 
#         model.fit(X_train, y_train)
#         train_score = model.score(X_train, y_train)
#         test_score = model.score(X_test, y_test) 
#         print("\n***** C = {} *****".format(i))
#         print("training accuracy: {}, ground truth (proportion y = 1): {}".format(train_score, y_train.mean()))
#         print("test accuracy: {}, ground truth (proportion y = 1): {}".format(test_score, y_test.mean()))
        
#         coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
#         coefs_pos = sorted(coefs, key = lambda x: x[1], reverse = True)
#         coefs_neg = sorted(coefs, key = lambda x: x[1], reverse = False)
#         coefs_abs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
#         coefs_nonzero = sum([x[1] != 0 for x in coefs])
#         coefs_zero = sum(x[1] == 0 for x in coefs)
#         print("Number of nonzero/zero coefficients: {}/{}".format(coefs_nonzero, coefs_zero))
#         print("Largest absolute coefficients:")
#         pprint(coefs_abs[:3])
#         print("Largest positive coefficients:")
#         pprint(coefs_pos[:3])
#         print("Largest negative coefficients:")
#         pprint(coefs_neg[:3])